# Test the edge configuration package locally

This notebook introduces you how to execute a created Pipeline in your local Python environment.  
During its execution, an `ImageSet` payload is created from images, as we discussed in the notebook [20-PreAndPostProcessing.ipynb](./20-PreAndPostProcessing.ipynb) and uses the Pipeline we created in notebook [30-CreatePipeline.ipynb](./30-CreatePipeline.ipynb).

The main goal to understand 
- how the test inputs are formed, 
- how the PythonComponent test environment is created
- how the GPURuntimeComponent test environment is created
- how to check the Pipeline behaviour

#### Create ImageSet input from a JPEG or PNG file

As we discussed in notebook [](./20-PreAndPostProcessing.ipynb), the payload of the Pipeline is an `ImageSet` in a dictionary, which will be processed by the Preprocessing step. For this reason, we create this payload from images in folder _../src/data/processed_ with the help of method defined in notebook [10-ObjectDetectionModel.ipynb](./10-ObjectDetectionModel.ipynb) and saved into Python script [payload.py](../src/preprocessing/payload.py).

In [ ]:
from pathlib import Path
import sys

sys.path.append('../src/')
from preprocessing.payload import create_imageset_dict

Create the required input format from an input JPG file using the method _create_imageset_dict_.

In [ ]:
data_path = Path("../data/processed/")
files = list(str(f) for f in data_path.rglob("*.jpg"))

pipeline_inputs = []
for image_file in files:
    image_set = create_imageset_dict(image_file, "BayerRG8")
    pipeline_inputs.append({ "vision_payload": image_set})

len(pipeline_inputs), pipeline_inputs[:1]

#### Test the pipeline configuration package locally
Take the created _pipeline_inputs_ to be tested as input of the `LocalPipelineRunner`, and run the pipeline with the test data.
The `LocalPipelineRunner` module is designed to be used __only__ inside a `with` block.

The `run_pipeline` method can be called multiple times but be aware that the internal state of the components is reset between the calls. Use batch input if you want to keep the state between inputs.

In [ ]:
from random import randint
from simaticai.testing.pipeline_runner import LocalPipelineRunner

image_on_edge_package = Path('../packages/BoardObjectDetection-edge_1.zip')
test_dir = Path("..").resolve() / "test"

pipeline_outputs = []
with LocalPipelineRunner(image_on_edge_package, test_dir) as runner:
    for i in range(10):
        pipeline_output = runner.run_pipeline(pipeline_inputs[randint(0, len(pipeline_inputs) - 1)])
        pipeline_outputs.append(pipeline_output)

pipeline_outputs

#### Test Pipeline step by step

In case of any issue occurs, there is an option to test the Pipeline by Components.  
Here we use the `run_component` method of `LocalPipelineRunner` instead of `run_pipeline`. In this case there are three Steps in the Pipeline, one for GPURuntimeComponent and two for PythonComponents. This way the inputs and outputs can be investigated independently, or you can execute only one step of the Pipeline, defining only the expected input of that Pipeline step.
The main difference between the two type of steps is the Python environment created for them to execute.  
While the test environment for a `PythonComponent` is created from the base environment by installing the defined Python packages defined in the requirements.txt of the component, the test environment for a `GPURuntimeComponent` installs only those Python packages, which are required to execute an ONNX model. 

In [ ]:
image_on_edge_package = Path('../packages/BoardObjectDetection-edge_1.zip')
test_dir = Path("..").resolve() / "test"

with LocalPipelineRunner(image_on_edge_package, test_dir) as runner:
    preprocessing_output  = runner.run_component("preprocessing", pipeline_inputs)

    detection_output = runner.run_component("detection", preprocessing_output)
    
    postprocessing_output = runner.run_component("postprocessing", detection_output)

preprocessing_output, detection_output, postprocessing_output

Once the expected outputs can be generated here, you are ready to step one further and give the Pipeline a try on `AI Inference Server`.
The main differences on the real device are
- underlying CPU, GPU configuration
- underlying Operating System
- used Python packages